In [ ]:
#import packages
from os import listdir
from nipype.interfaces.io import DataSink, SelectFiles # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, JoinNode        # pypeline engine

from nipype.interfaces.fsl.model import Randomise, GLM, Cluster
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.fsl.utils import ImageMeants, Merge

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
studyhome = '/Users/catcamacho/Box/BABIES'
raw_data = studyhome + '/raw'
preproc_dir = studyhome + '/processed/preproc'
output_dir = studyhome + '/processed/analysis'
workflow_dir = studyhome + '/workflows'
roi_dir = studyhome + '/ROIs'

subjects_list = ['021-BABIES-T1','033x-BABIES-T1'] #listdir(raw_data)
template_brain = studyhome + '/templates/T2w_BABIES_template_2mm.nii'

# ROIs for connectivity analysis
Lamyg = roi_dir + '/Lamyg_4mm.nii'
Ramyg = roi_dir + '/Ramyg_4mm.nii'
roi_files = [roi_dir + '/L_amyg_4mm.nii',roi_dir + '/R_amyg_4mm.nii']

ROIs = [Lamyg, Ramyg]


In [ ]:
## File handling
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subjects_list)


# Data grabber- select fMRI and ROIs
templates = {'filt_func': output_dir + '/bp_filtered_func/_subject_id_{subject_id}/func_filtered.nii',
             'orig_func': output_dir + '/denoised_func/_subject_id_{subject_id}/denoised_func.nii', 
             'rois':ROIs}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [ ]:
## Seed-based level 1

# Extract ROI timeseries
def pulltimeseries(roimask,func):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nipype.interfaces.fsl.maths import ApplyMask
    from nibabel import load
    from numpy import mean, std, savetxt
    from os import path
    
    roimaskfile = path.basename(roimask)
    roimaskname = roimaskfile.replace('.nii','')
    datapath = path.dirname(func)
    subID = datapath[-3:]
    
    roi = 'roi.nii'
    am = ApplyMask()
    am.inputs.mask_file = roimask
    am.inputs.in_file = func
    am.inputs.out_file = roi
    am.run()

    subnifti = load(roi)
    subdata = subnifti.get_data()
    roitimeseries = mean(subdata,axis=3)
    
    roitextfile = roimaskname + '_' + subID + '_timeseries.txt'
    savetxt(roitextfile, roitimeseries, delimiter=' ')
    roi_ts = path.abspath(roitextfile)
    return(roi_ts)
        
pullROItimeseries = Node(name='pullROItimeseries',
                         interface=Function(input_names=['roimask','func'],
                                            output_names=['roi_ts'],
                                            function=pulltimeseries),
                         iterfield=['roimask'])

ROI_timeseries = Node(ImageMeants(), name='ROI_timeseries', iterfield='mask')

# model ROI connectivity
glm = Node(GLM(), name='glm', iterfield='design')

In [ ]:
## Level 2

# merge param estimates across all subjects per seed
merge = JoinNode(Merge(dimension='t'), 
                 joinfield='in_files', 
                 joinsource='glm',
                 iterfield='in_files',
                 name='merge')

# FSL randomise for higher level analysis
highermodel = Node(Randomise(tfce=True,
                             raw_stats_imgs= True,
                             design_mat=group_mat),
                   iterfield='in_file',
                   name = 'highermodel')

## Cluster results

# make binary masks of sig clusters
binarize = Node(Binarize(min=0.95, max=1.0), name='binarize')

# clusterize and extract cluster stats/peaks
clusterize = Node(Cluster(threshold=2, 
                          out_index_file=True, 
                          out_localmax_txt_file=True, 
                          out_localmax_vol_file=True,
                          out_max_file=True), 
                  name='clusterize')

# make pictures if time

In [ ]:
sbc_workflow = Workflow(name='sbc_workflow')
sbc_workflow.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                      (selectfiles,ROI_timeseries,[('orig_func','in_file')]),
                      (selectfiles,ROI_timeseries,[('rois','mask')]),
                      (ROI_timeseries,glm,[('out_file','design')]),
                      (glm,merge,[('out_cope','in_files')]),
                      (merge,highermodel,[('merged_file','in_file')]),
                      
                      (ROI_timeseries, datasink, [('out_file','roi_ts')]),
                      (glm,datasink,[('out_cope','seed_copes')]),
                      ()
                     ])
sbc_workflow.base_dir = workflow_dir
sbc_workflow.write_graph(graph2use='flat')
sbc_workflow.run('MultiProc', plugin_args={'n_procs': proc_cores})